# Post Deployment Language Switching QA

[Task](https://phabricator.wikimedia.org/T275762)
|
[Schema](https://gerrit.wikimedia.org/r/plugins/gitiles/schemas/event/secondary/+/refs/heads/master/jsonschema/analytics/legacy/universallanguageselector/current.yaml)

# Contents
1. [Language Links in the Sidebar](#Language-Links-in-the-Sidebar)
2. [Language Switcher Button](#Language-Switcher-Button)
3. [Language Button Context Events](#Language-Button-Context-Events)

In [6]:
shhh <- function(expr) suppressPackageStartupMessages(suppressWarnings(suppressMessages(expr)))
shhh({
    library(tidyverse); 
    library(lubridate); 
    library(scales);
    library(magrittr); 
    library(dplyr)
})


## Language Links in the Siderbar

[Instrumentation Task](https://phabricator.wikimedia.org/T275762)

Instrumentation Notes:
- Instrumentation limited to legacy sidebar in modern Vector.
- Logged as `event.context = 'languages-list'`

In [2]:
query <- 
"
SELECT
    TO_DATE(dt) AS `date`,
    wiki,
    event.web_session_id,
    event.usereditbucket,
    event.timetochangelanguage,
    event.isanon,
    event.interfacelanguage,
    event.contentlanguage,
    event.selectedinterfacelanguage,
    Count(*) AS n_events
FROM event.universallanguageselector
WHERE
    year = 2021
    AND ((Month = 04 AND DAY > 26) OR (MONTH = 05))
    AND event.context = 'languages-list'
GROUP BY
    TO_DATE(dt),
    wiki,
    event.web_session_id,
    event.usereditbucket,
    event.timetochangelanguage,
    event.isanon,
    event.interfacelanguage,
    event.contentlanguage,
    event.selectedinterfacelanguage
"

In [3]:
lang_link_events <-  wmfdata::query_hive(query)

Don't forget to authenticate with Kerberos using kinit



In [4]:
lang_link_events$date <- as.Date(lang_link_events$date)

## Daily Langauge Link Events

In [5]:
lang_link_events_daily <- lang_link_events %>%
    group_by(date) %>%
    summarize(n_events = sum(n_events),
             n_sessions = n_distinct(web_session_id))

lang_link_events_daily

`summarise()` ungrouping output (override with `.groups` argument)



date,n_events,n_sessions
<date>,<int>,<int>
2021-04-28,791,636
2021-04-29,46934,33173
2021-04-30,274519,177001
2021-05-01,243709,151370
2021-05-02,258630,161650
2021-05-03,303583,195465
2021-05-04,300642,194931
2021-05-05,295520,191876
2021-05-06,291788,190268


We start recording events on 28 April 2021. There are an average 176,827 sessions per day including sessions by both logged in and logged out users. No unexpected spikes or drops so far.

## Clicks per session

In [ ]:
Check to make sure there are duplicate session id. Some sessions should have more than one click event.

In [9]:
length(unique(lang_link_events$web_session_id)) == nrow(lang_link_events)

[1] FALSE

In [12]:
lang_link_events_persession <- lang_link_events %>%
    group_by(isanon) %>%
    summarize(avg_clicks = mean(n_events),
              max_clicks = max(n_events),
              min_clciks = min(n_events))

lang_link_events_persession

`summarise()` ungrouping output (override with `.groups` argument)



isanon,avg_clicks,max_clicks,min_clciks
<chr>,<dbl>,<int>,<int>
false,1.000327,2,1
true,1.000169,4,1


## By Logged In Status

In [13]:
lang_link_events_isanon <- lang_link_events %>%
    group_by(isanon) %>%
    summarize(n_events = sum(n_events),
             n_sessions = n_distinct(web_session_id))

lang_link_events_isanon 

`summarise()` ungrouping output (override with `.groups` argument)



isanon,n_events,n_sessions
<chr>,<int>,<int>
false,113161,46278
true,3568719,2058676


97.8% of all sessions with clicks to the language links are by logged out users. That's high but expected because instrumentation was limited to legacy sidebar in modern Vector (not from legacy or other skins such as timeless). The new language switching functionality was made available to all logged-in users opted into the latest version of the Vector skin.

Legacy sidebar in modern Vector would mostly appear to logged-out users on test wikis where Vector is deployed as default.


## By Test Wiki

In [36]:
# events and sessions that include link to language link by test wiki
lang_link_events_testwiki <- lang_link_events %>%
    filter(wiki %in% c('frwiktionary', 'hewiki', 'ptwikiversity', 'frwiki', 
    'euwiki', 'fawiki', 'ptwiki', 'kowiki', 'trwiki', 'srwiki', 'bnwiki', 'dewikivoyage', 'vecwiki' )) %>%
    group_by(wiki) %>%
    summarize(n_events = sum(n_events),
             n_sessions = n_distinct(web_session_id))

lang_link_events_testwiki


`summarise()` ungrouping output (override with `.groups` argument)



wiki,n_events,n_sessions
<chr>,<int>,<int>
bnwiki,8451,5726
dewikivoyage,755,604
euwiki,31609,22351
fawiki,200781,110092
frwiki,1933743,1091442
frwiktionary,21307,13350
hewiki,173526,100165
kowiki,146461,81522
ptwiki,555228,328675


In [39]:
# events and sessions that include link to language link by test wiki category and logged-in status
lang_link_events_testwiki_isanon <- lang_link_events %>%
    mutate(istestwiki = ifelse(wiki %in% c('frwiktionary', 'hewiki', 'ptwikiversity', 'frwiki', 
    'euwiki', 'fawiki', 'ptwiki', 'kowiki', 'trwiki', 'srwiki', 'bnwiki', 'dewikivoyage', 'vecwiki' ), 'test_wiki', 'non_test_wiki')) %>%
    group_by(istestwiki, isanon) %>%
    summarize(n_events = sum(n_events),
             n_sessions = n_distinct(web_session_id))

lang_link_events_testwiki_isanon

`summarise()` regrouping output by 'istestwiki' (override with `.groups` argument)



istestwiki,isanon,n_events,n_sessions
<chr>,<chr>,<int>,<int>
non_test_wiki,false,3540,1422
non_test_wiki,true,147,80
test_wiki,false,102371,41836
test_wiki,true,3325559,1922334


Almost all of the events recorded to date (99%) have been on test wikis. This is expected as the new language switcher button was deployed to all users opt'd in to the modern vector on all non test wikis. Users with modern vector on the test wikis have not been shown the new language switcher and still shown the language links in the sidebar.

On non test wikis, the majority (94.67%) of sessions with clicks to the language list on modern vector come from logged-in users. Need to confirm if it's possible to have language link in sidebar if you are logged-in, on modern vector and on a non test wiki. 


## By User Edit Bucket

In [30]:
logged_in_editcount <- lang_link_events %>%
    filter(isanon == 'false') %>%
    group_by(usereditbucket) %>%
    summarize(n_events = sum(n_events),
             n_sessions = n_distinct(web_session_id))

logged_in_editcount

`summarise()` ungrouping output (override with `.groups` argument)



usereditbucket,n_events,n_sessions
<chr>,<int>,<int>
0 edits,20092,10961
1-4 edits,11510,5643
100-999 edits,17264,6533
1000+ edits,31482,8932
5-99 edits,25556,11332
NULL,7,5


In [31]:
logged_out_editcount <- lang_link_events %>%
    filter(isanon == 'true') %>%
    group_by(usereditbucket) %>%
    summarize(n_events = sum(n_events),
             n_sessions = n_distinct(web_session_id))

logged_out_editcount

`summarise()` ungrouping output (override with `.groups` argument)



usereditbucket,n_events,n_sessions
<chr>,<int>,<int>
5-99 edits,1,1
NULL,3325705,1922412


There are just a few instances (under 0.01%) of the event.usereditbucket field being populated for logged out users and recorded as NULL for logged-in users. Further investigation might be needed; however, the numbers of these events is not high enough to skew the data.

## By Final Language

In [43]:
# test that you can switch from one language to the next.
top_final_languages <- lang_link_events %>%
    mutate(all_sessions = n_distinct(web_session_id)) %>%
    group_by(selectedinterfacelanguage) %>%
    summarize(n_sessions = n_distinct(web_session_id),
             pct_sessions = n_sessions/all_sessions) %>%
    distinct() %>%
    arrange(desc(n_sessions))

head(top_final_languages )

`summarise()` regrouping output by 'selectedinterfacelanguage' (override with `.groups` argument)



selectedinterfacelanguage,n_sessions,pct_sessions
<chr>,<int>,<dbl>
en,1443105,0.73470332
es,128358,0.06534871
de,125061,0.06367016
it,65010,0.03309743
ar,64408,0.03279094
ru,52428,0.02669177


The most frequent language switches are to english (73% of sessions) followed by spanish (6.5%), and german (6.3%).

## By Initial Language 

In [48]:

top_initial_languages <- lang_link_events %>%
    mutate(all_sessions = n_distinct(web_session_id)) %>%
    group_by(interfacelanguage, contentlanguage) %>%
    summarize(n_sessions = n_distinct(web_session_id),
             pct_sessions = n_sessions/all_sessions) %>%
    distinct() %>%
    arrange(desc(n_sessions))

head(top_initial_languages )

`summarise()` regrouping output by 'interfacelanguage', 'contentlanguage' (override with `.groups` argument)



interfacelanguage,contentlanguage,n_sessions,pct_sessions
<chr>,<chr>,<int>,<dbl>
fr,fr,1103742,0.56192925
pt,pt,328012,0.16699513
tr,tr,150977,0.07686433
fa,fa,109989,0.05599681
he,he,100125,0.05097493
ko,ko,81429,0.04145655


The interfacelanguage and contentlanguage will usually be the same and should match for most instances, which is confirmed here.

The top initial languages all from test wikis, which is expected since the language links are still shown to all logged-in and logged-out users on modern vector. The new language switcher, which replaces the lang links with a button, are show to all logged-in users opt'd into modern vector on non-test wikis. 

In [ ]:
## Most Frequent Switch Types


In [49]:
top_final_languages <- lang_link_events %>%
    mutate(all_sessions = n_distinct(web_session_id)) %>%
    group_by(interfacelanguage, selectedinterfacelanguage) %>%
    summarize(n_sessions = n_distinct(web_session_id),
             pct_sessions = n_sessions/all_sessions) %>%
    distinct() %>%
    arrange(desc(n_sessions))

head(top_final_languages )

`summarise()` regrouping output by 'interfacelanguage', 'selectedinterfacelanguage' (override with `.groups` argument)



interfacelanguage,selectedinterfacelanguage,n_sessions,pct_sessions
<chr>,<chr>,<int>,<dbl>
fr,en,775769,0.39495398
pt,en,264373,0.13459570
tr,en,118824,0.06049483
fa,en,95622,0.04868239
fr,de,94915,0.04832245
he,en,84668,0.04310557


39% of all sessions are switches are from French to English. 

# Language Switcher Button

[Instrumentation Task](https://phabricator.wikimedia.org/T281928)

- Instrumented as `event.action="compact-language-links-open"`
- Logged when a user open the new language switcher button. 
- The first click and subsequent clicks should be recorded per https://gerrit.wikimedia.org/r/c/mediawiki/extensions/UniversalLanguageSelector/+/688994/
- Change merged on May 11th and backported/deployed on May 12th.
- There should not be any clicks to the Language Switcher button by logged-in users on test wikis or logged-out users, as the new lanuage button has not been deployed to these wikis or these users.
- Only deployed as logged-in users on the latest vector on a non test wiki

In [191]:
query <- 
"
SELECT
    TO_DATE(dt) AS `date`,
    wiki,
    event.web_session_id,
    event.usereditbucket,
    event.skin,
    event.skinVersion,
    event.timetochangelanguage,
    event.isanon,
    event.context,
    event.interfacelanguage,
    event.contentlanguage,
    event.selectedinterfacelanguage,
    Count(*) AS n_events
FROM event.universallanguageselector
WHERE
    year = 2021
    AND month >= 05
    AND event.action = 'compact-language-links-open'
    AND useragent.is_bot = false
GROUP BY
    TO_DATE(dt),
    wiki,
    event.web_session_id,
    event.usereditbucket,
    event.skin,
    event.skinVersion,
    event.timetochangelanguage,
    event.isanon,
    event.context,
    event.interfacelanguage,
    event.contentlanguage,
    event.selectedinterfacelanguage
"

In [192]:
lang_button_events <-  wmfdata::query_hive(query)

Don't forget to authenticate with Kerberos using kinit



## Daily Lang Button Click Events

In [193]:
lang_button_events_daily <- lang_button_events %>%
    group_by(date) %>%
    summarize(n_events = sum(n_events),
             n_sessions = n_distinct(web_session_id))

lang_button_events_daily

`summarise()` ungrouping output (override with `.groups` argument)



date,n_events,n_sessions
<chr>,<int>,<int>
2021-05-12,12,10
2021-05-13,194,159
2021-05-14,1317,1147
2021-05-15,1159,928
2021-05-16,1358,1042
2021-05-17,32668,28541
2021-05-18,90811,78945
2021-05-19,89299,77108
2021-05-20,87644,75820


- Confirmed we start recording events on May 12th following backport of the change, starting with just a few events and then more as change rolled out to additional wikis.
-  There is an expected higher number of events than sessions, which is expected as a session can include multiple clicks to open the language button.


# By Context and Vector Version

In [194]:
lang_button_events_bycontext <- lang_button_events %>%
# date new instrumentation was added
    filter(date >= '2021-06-08')%>%
    group_by(skinversion, context) %>%
    summarize(n_events = sum(n_events),
             n_sessions = n_distinct(web_session_id))

lang_button_events_bycontext

`summarise()` regrouping output by 'skinversion' (override with `.groups` argument)



skinversion,context,n_events,n_sessions
<chr>,<chr>,<int>,<int>
latest,header,4892,1480
latest,other,34860,28752
legacy,other,373435,305984
NULL,NULL,36732,31106


In [ ]:
Conirmed we are only recording new button clicks on the latest vector.

## Double Check to make sure there are some sessions with more than one event

In [195]:
length(unique(lang_button_events$web_session_id)) == nrow(lang_button_events)

[1] FALSE

## Lang Button Events Per Wiki

In [196]:

lang_button_events_bywiki <- lang_button_events %>%
    group_by(wiki) %>%
    summarize(n_events = sum(n_events),
             n_sessions = n_distinct(web_session_id)) %>%
    arrange(desc(n_sessions))

head(lang_button_events_bywiki)

`summarise()` ungrouping output (override with `.groups` argument)



wiki,n_events,n_sessions
<chr>,<int>,<int>
enwiki,1139559,898960
ruwiki,155451,121934
dewiki,136256,113844
eswiki,105962,87827
frwiki,93685,75916
zhwiki,70792,57560


Almost half (49.7%) of sessions with a click to the language button were recorded on English Wikipedia, followed by Russian, German and Spanish Wikipedia. Note: The language switcher button is currently not availabe to logged-in users on test wikis.



# Lang Button Event on Test Wikis

In [197]:

lang_button_events_bytestwiki <- lang_button_events %>%
    filter(wiki %in% c('frwiktionary', 'hewiki', 'ptwikiversity', 'frwiki', 
    'euwiki', 'fawiki', 'ptwiki', 'kowiki', 'trwiki', 'srwiki', 'bnwiki', 'dewikivoyage', 'vecwiki' ))  %>%
    group_by(wiki) %>%
    summarize(n_events = sum(n_events),
             n_sessions = n_distinct(web_session_id)) %>%
    arrange(desc(n_sessions))

lang_button_events_bytestwiki

`summarise()` ungrouping output (override with `.groups` argument)



wiki,n_events,n_sessions
<chr>,<int>,<int>
frwiki,93685,75916
ptwiki,25703,21272
trwiki,22477,18339
srwiki,14190,12135
kowiki,10200,8362
fawiki,9331,7527
hewiki,6848,5785
euwiki,2062,1727
frwiktionary,1392,1191


About 8.4% of all sessions were recorded on the early adopter wikis. 

I reviewed the number of sessions on test wikis by logged-in status to determine if these were mostly logged-out or logged-in users.

In [198]:
# events and sessions that include link to language link by test wiki category and logged-in status
lang_button_events_testwiki_isanon <- lang_button_events %>%
    mutate(istestwiki = ifelse(wiki %in% c('frwiktionary', 'hewiki', 'ptwikiversity', 'frwiki', 
    'euwiki', 'fawiki', 'ptwiki', 'kowiki', 'trwiki', 'srwiki', 'bnwiki', 'dewikivoyage', 'vecwiki' ), 'test_wiki', 'non_test_wiki')) %>%
    group_by(istestwiki, isanon) %>%
    summarize(n_events = sum(n_events),
             n_sessions = n_distinct(web_session_id))

lang_button_events_testwiki_isanon 

`summarise()` regrouping output by 'istestwiki' (override with `.groups` argument)



istestwiki,isanon,n_events,n_sessions
<chr>,<chr>,<int>,<int>
non_test_wiki,false,76240,45477
non_test_wiki,true,1930915,1566932
test_wiki,false,6865,5030
test_wiki,true,180522,148621


0.29% of sessions with a click to the lang switcher button came from logged-in users on the early adopter wikis and 8.1% of sessions by logged-out users. 


## By anon status

In [74]:
lang_button_events_byanon <- lang_button_events %>%
    group_by(isanon) %>%
    summarize(n_events = sum(n_events),
             n_sessions = n_distinct(web_session_id)) 

head(lang_button_events_byanon)

`summarise()` ungrouping output (override with `.groups` argument)



isanon,n_events,n_sessions
<chr>,<int>,<int>
false,22534,14454
true,592126,494268


The majority of sessions with clicks to the language button are by logged out users (97.2%) 

## Clicks per session

In [49]:
lang_button_events_persession <- lang_button_events %>%
    group_by(isanon) %>%
    summarize(avg_clicks = mean(n_events),
              max_clicks = max(n_events),
              min_clciks = min(n_events))

lang_button_events_persession

`summarise()` ungrouping output (override with `.groups` argument)



isanon,avg_clicks,max_clicks,min_clciks
<chr>,<dbl>,<int>,<int>
false,1.496471,84,1
true,1.185743,798,1


Most sessions include between 1 to 2 clicks to the language switcher button.

Note: There are some sessions by anon users with over 600 clicks, which is is likely automated traffic from bots and can be filtered out.

## User Edit Bucket

In [36]:
logged_in_editcount <- lang_button_events %>%
    filter(isanon == 'false') %>%
    group_by(usereditbucket) %>%
    summarize(n_events = sum(n_events),
             n_sessions = n_distinct(web_session_id))

logged_in_editcount

`summarise()` ungrouping output (override with `.groups` argument)



usereditbucket,n_events,n_sessions
<chr>,<int>,<int>
0 edits,5936,4666
1-4 edits,2404,1764
100-999 edits,3215,1888
1000+ edits,5293,2628
5-99 edits,5156,3202


In [37]:
logged_out_editcount <- lang_button_events %>%
    filter(isanon == 'true') %>%
    group_by(usereditbucket) %>%
    summarize(n_events = sum(n_events),
             n_sessions = n_distinct(web_session_id))

logged_out_editcount

`summarise()` ungrouping output (override with `.groups` argument)



usereditbucket,n_events,n_sessions
<chr>,<int>,<int>
NULL,578951,483477


Confirmed that we are not recording an edit count for logged out users as expected.

## Content Language

In [38]:

top_content_languages <- lang_button_events %>%
    mutate(all_sessions = n_distinct(web_session_id)) %>%
    group_by(interfacelanguage, contentlanguage) %>%
    summarize(n_sessions = n_distinct(web_session_id),
             pct_sessions = n_sessions/all_sessions) %>%
    distinct() %>%
    arrange(desc(n_sessions))

head(top_content_languages )

`summarise()` regrouping output by 'interfacelanguage', 'contentlanguage' (override with `.groups` argument)



interfacelanguage,contentlanguage,n_sessions,pct_sessions
<chr>,<chr>,<int>,<dbl>
en,en,257528,0.51773060
ru,ru,35846,0.07206428
de,de,31134,0.06259135
es,es,24863,0.04998422
fr,fr,21550,0.04332381
it,it,13693,0.02752821


The top current content and interface lang settings are english, russian, german and spanish, which fits with the top wikis we see where the language button switch clicks occur.

## Event Context

In [39]:

lang_button_events_context <- lang_button_events %>%
    group_by(context) %>%
    summarize(n_events = sum(n_events),
             n_sessions = n_distinct(web_session_id))

lang_button_events_context

`summarise()` ungrouping output (override with `.groups` argument)



context,n_events,n_sessions
<chr>,<int>,<int>
NULL,600955,497417


We are not recording `event.context` for these events as expected. 

Note: This was changed with a fix deployed on 6 June 2021.

## Time to Change Language

In [40]:

lang_button_events_time <- lang_button_events %>%
    summarise(avg_time = mean(timetochangelanguage),
             median_time = median(timetochangelanguage),
             max_time = max(timetochangelanguage),
             min_time = min(timetochangelanguage))

lang_button_events_time

Warning message in mean.default(timetochangelanguage):
“argument is not numeric or logical: returning NA”
Warning message in mean.default(sort(x, partial = half + 0L:1L)[half + 0L:1L]):
“argument is not numeric or logical: returning NA”


avg_time,median_time,max_time,min_time
<dbl>,<dbl>,<chr>,<chr>
NA,NA,NULL,NULL


Note: We don't record time to change language for the initial click to the button but we record when the user actually changes languages.

## Confirm we can identify language switches for sessions where button is clicked

In [130]:
#rough query to confirm approach
# will be refine in analysis
query <- 
"
-- sessions where lang button was selected
WITH button AS (
SELECT
    MIN(TO_DATE(dt)) as button_date,
    event.web_session_id as session_id,
    wiki as wiki
FROM event.universallanguageselector
WHERE
    year = 2021
    AND month >= 05
    AND useragent.is_bot = false
    AND event.action = 'compact-language-links-open'
GROUP BY 
    event.web_session_id,
    wiki
),

lang_switches AS (
  SELECT
    TO_DATE(dt) as switch_date,
    event.web_session_id as session_id,
    event.context as switch_context,
    wiki as wiki
FROM event.universallanguageselector
WHERE
    year = 2021
    AND month >= 05
    AND useragent.is_bot = false
    AND event.action = 'language-change'
)

SELECT
    button.button_date,
    lang_switches.switch_date,
    button.session_id,
    button.wiki,
-- sessions with lang switch that occured after button clicks
    IF(lang_switches.session_id IS NOT NULL AND switch_date >= button_date, 1, 0) AS language_switch,
    lang_switches.switch_context
FROM button
LEFT JOIN lang_switches ON
    button.session_id = lang_switches.session_id AND
    button.wiki = lang_switches.wiki
    
"

In [111]:
lang_button_switch_events <-  wmfdata::query_hive(query)

Don't forget to authenticate with Kerberos using kinit



In [112]:
#reformat to date format
lang_button_switch_events$button_date <- as.Date(lang_button_switch_events$button_date, format = "%Y-%m-%d")
lang_button_switch_events$switch_date <- as.Date(lang_button_switch_events$switch_date, format = "%Y-%m-%d")

## Lang Button Sessions with Language Switch

In [114]:
lang_button_switch_events_nsessions <- lang_button_switch_events %>%
    mutate(language_switch = ifelse(language_switch == 0, "no_switch", "switch")) %>%
    group_by(language_switch) %>%
    summarise(n_sessions = n_distinct(session_id))

lang_button_switch_events_nsessions

`summarise()` ungrouping output (override with `.groups` argument)



language_switch,n_sessions
<chr>,<int>
no_switch,1485327
switch,53679


About 3.4% of sessions where the language button was clicked was followed by an event to switch the language. 

## Lang Button Sessions with Switch by Context

In [115]:
lang_button_switch_events_nsessions <- lang_button_switch_events %>%
    filter(language_switch == 1) %>%
    group_by(switch_context) %>%
    summarise(n_sessions = n_distinct(session_id))

lang_button_switch_events_nsessions

`summarise()` ungrouping output (override with `.groups` argument)



switch_context,n_sessions
<chr>,<int>
interface,485
languages-list,53205


Confirmed that we are only recording `event.context` for langauge switches. There are two types of switches we are recording: 'interface" and 'languages-list'.

Almost all (99%) of sessions that switched languages after clicking the language button were recorded with `event.context = 'languages-list'`.  

Note: We also use 'languages-list' to identify clicks to the sidebar. I'll need to use sessions that opened language switcher button to identify those that recieved the new treatment.

# Language Button Context Events

Change deployed on 8 June 2021
Rechecked data to confirm that events are recording as expected and that were able to distinguish between the following three event types:

* People selecting language button on top of the page
* People that are in the control bucket clicking the "more" button
* People that have opted out of modern vector clicking the more button

Update: Fix deployed on 10 June 2021 to address bug identified in post deployment QA, where links to switch languages after clicking the langauge switcher button were not instrumented. Data updated below to reflect changes following this fix.

Note on instrumentation:
If the user has opened the language switcher in the header and switched language, we should two events with the following properties:

1. event.web_session_id="foo" event.action="compact-language-links-open" event.context="header" event.skinversion="latest"
2. event.web_session_id="foo" event.action="language-change" event.context="interface"`


## By Skin Type

In [200]:
lang_button_switch_events_byskin <- lang_button_events %>%
    filter(date >= '2021-06-11') %>% # date fix deployed
    group_by(date, skin, skinversion) %>%
    summarise(n_sessions = n_distinct(web_session_id))

lang_button_switch_events_byskin 

`summarise()` regrouping output by 'date', 'skin' (override with `.groups` argument)



date,skin,skinversion,n_sessions
<chr>,<chr>,<chr>,<int>
2021-06-11,NULL,NULL,18
2021-06-11,vector,latest,5467
2021-06-11,vector,legacy,54874
2021-06-12,NULL,NULL,14
2021-06-12,vector,latest,4363
2021-06-12,vector,legacy,45448
2021-06-13,NULL,NULL,5
2021-06-13,vector,latest,5053
2021-06-13,vector,legacy,49477


We are recording two skin types: 'NULL and 'vector' and recording both skin version types for vector:  'latest' and 'legacy'. I'm assuming NULL counts as all skin types not as vector. It's not needed for the AB test but I recommend revising to clarify the specific non-vector skin types at some point.

In [201]:
# By Context Field

In [202]:
lang_button_switch_events_bycontext <- lang_button_events %>%
    filter(date >= '2021-06-11') %>% # date fix deployed
    group_by(context, skin, skinversion) %>%
    summarise(n_sessions = n_distinct(web_session_id))

lang_button_switch_events_bycontext 

`summarise()` regrouping output by 'context', 'skin' (override with `.groups` argument)



context,skin,skinversion,n_sessions
<chr>,<chr>,<chr>,<int>
header,vector,latest,834
NULL,NULL,NULL,39
other,vector,latest,13957
other,vector,legacy,149212


* Confirmed that with new instrumentation we can differentiate the following event types:
  * People on vector selecting language button on top of the page:  `event.action = 'compact-language-links-open'`; `event.context = 'header'`; `event.skinVersion = 'latest'`
  * People on vector that are in the control bucket clicking the " N more" button in the sidebar: `event.action = 'compact-language-links-open'` and `event.context = 'other'`, `event.skinVersion = 'latest'`
  * People that have opted out of modern vector clicking the more button: `event.context = 'NULL'`; `event.skinVersion = 'NULL'`


We are only recording clicks to the new button in the header on the latest vector as expected. 

There are clicks to the N More button by both legacy and latest as expected. The new language button was not deployed to logged-in users on test wikis so they would still have clicks to the N more button on the latest vector. When the AB test run, only users in the control group will be able to access the N more button on the latest vector.

Note context value events are only set for user on vector; however, we can identify click from other skins to the N More button by the lack of values (`event.context = NULL`)



## By logged-in and logged-out status

In [203]:
lang_button_switch_events_byusertype <- lang_button_events %>%
    filter(date >= '2021-06-11') %>% # date fix deployed
    group_by(context, skin, skinversion, isanon) %>%
    summarise(n_sessions = n_distinct(web_session_id))

lang_button_switch_events_byusertype

`summarise()` regrouping output by 'context', 'skin', 'skinversion' (override with `.groups` argument)



context,skin,skinversion,isanon,n_sessions
<chr>,<chr>,<chr>,<chr>,<int>
header,vector,latest,false,831
header,vector,latest,true,3
NULL,NULL,NULL,false,1
NULL,NULL,NULL,true,38
other,vector,latest,false,415
other,vector,latest,true,13543
other,vector,legacy,false,4156
other,vector,legacy,true,145100


## Totals by Anon

In [204]:
lang_button_events_byusertype_all <- lang_button_events %>%
    filter(date >= '2021-06-11',
          context == 'header') %>% # date fix deployed
    group_by(isanon) %>%
    summarize(n_events = sum(n_events),
             n_sessions = n_distinct(web_session_id)) 

head(lang_button_events_byusertype_all)

`summarise()` ungrouping output (override with `.groups` argument)



isanon,n_events,n_sessions
<chr>,<int>,<int>
false,2524,831
true,5,3


Confirmed we are only recording clicks to the new language button by logged-in users. All new button clicks recorded so far have been on latest vector. 

Update: Identified three header sessions by logged-out users - Need to investigate further.

### Investigate logged-out new lang button events

In [205]:
lang_button_events_isanon <- lang_button_events %>%
    filter(date >= '2021-06-11',
          context == 'header',
        isanon == "true")

lang_button_events_isanon 

date,wiki,web_session_id,usereditbucket,skin,skinversion,timetochangelanguage,isanon,context,interfacelanguage,contentlanguage,selectedinterfacelanguage,n_events
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>
2021-06-11,testwiki,bc29f4fc89d59f4bc99c,NULL,vector,latest,NULL,true,header,en,en,NULL,1
2021-06-11,bnwiki,4fa77beb78338661d19c,NULL,vector,latest,NULL,true,header,bn,bn,NULL,1
2021-06-13,testwiki,1c87469b4eb16ead0427,NULL,vector,latest,NULL,true,header,en,en,NULL,3


In [ ]:
Two of these events occured on testwiki and one on bwiki, which is one of the early adopter wikis.


# By Test Wiki

In [206]:
lang_button_switch_events_byisanon_testwiki <- lang_button_events %>%
    filter(wiki %in% c('frwiktionary', 'hewiki', 'ptwikiversity', 'frwiki', 
    'euwiki', 'fawiki', 'ptwiki', 'kowiki', 'trwiki', 'srwiki', 'bnwiki', 'dewikivoyage', 'vecwiki' ), 
        date >= '2021-06-11') %>% # date fix deployed
    group_by(context, skin, skinversion, isanon) %>%
    summarise(n_sessions = n_distinct(web_session_id))

lang_button_switch_events_byisanon_testwiki

`summarise()` regrouping output by 'context', 'skin', 'skinversion' (override with `.groups` argument)



context,skin,skinversion,isanon,n_sessions
<chr>,<chr>,<chr>,<chr>,<int>
header,vector,latest,true,1
NULL,NULL,NULL,true,3
other,vector,latest,false,414
other,vector,latest,true,13538
other,vector,legacy,false,98


We have recorded one events on the early adopter wiki, which is not expected. The bnwiki event identified above, which should not happen.

Numbers appear as expected. There are a limited number of events recorded on legacy vector since the new skin is deployed as opt-out.

# By Non Test Wiki

In [207]:
lang_button_switch_events_bynontest <- lang_button_events %>%
    filter(skinversion == 'latest',
           skin == 'vector',
           date >= '2021-06-11',
        !wiki %in% c('frwiktionary', 'hewiki', 'ptwikiversity', 'frwiki', 
    'euwiki', 'fawiki', 'ptwiki', 'kowiki', 'trwiki', 'srwiki', 'bnwiki', 'dewikivoyage', 'vecwiki' ), 
        date >= '2021-06-08') %>% # date fix deployed
    group_by(context, skin, skinversion, isanon) %>%
    summarise(n_sessions = n_distinct(web_session_id),
             n_events = sum(n_events))

lang_button_switch_events_bynontest

`summarise()` regrouping output by 'context', 'skin', 'skinversion' (override with `.groups` argument)



context,skin,skinversion,isanon,n_sessions,n_events
<chr>,<chr>,<chr>,<chr>,<int>,<int>
header,vector,latest,false,831,2524
header,vector,latest,true,2,4
other,vector,latest,false,1,1
other,vector,latest,true,5,5


On non-test wikis, we should only be recording clicks to the new button by logged-in users. There are two events by logged-out users which occurs on test wiki, see above.


## Content Language Check

In [162]:

top_content_languages <- lang_button_events %>%
    filter(date >= '2021-06-11',
              context == 'header') %>%
    mutate(all_sessions = n_distinct(web_session_id)) %>%
    group_by(interfacelanguage, contentlanguage) %>%
    summarize(n_sessions = n_distinct(web_session_id),
             pct_sessions = n_sessions/all_sessions) %>%
    distinct() %>%
    arrange(desc(n_sessions))

head(top_content_languages)

`summarise()` regrouping output by 'interfacelanguage', 'contentlanguage' (override with `.groups` argument)



interfacelanguage,contentlanguage,n_sessions,pct_sessions
<chr>,<chr>,<int>,<dbl>
en,en,85,0.29513889
ja,ja,20,0.06944444
zh-tw,zh,20,0.06944444
de,de,14,0.04861111
ru,ru,12,0.04166667
ca,ca,11,0.03819444


No test wikis and larger size non-test wikis are listed as the top content langauges where a new search button was clicked.

## Edit Count

In [209]:

lang_button_byeditcount <- lang_button_events %>%
    filter(date >= '2021-06-11',
              context == 'header') %>%
    group_by(usereditbucket, isanon) %>%
    summarize(n_sessions = n_distinct(web_session_id))

lang_button_byeditcount

`summarise()` regrouping output by 'usereditbucket' (override with `.groups` argument)



usereditbucket,isanon,n_sessions
<chr>,<chr>,<int>
0 edits,false,93
1-4 edits,false,64
100-999 edits,false,135
1000+ edits,false,360
5-99 edits,false,181
NULL,true,3


There are three NULL events recorded. These are all for the logged-out user events with clicks to the header identified above.

There appears to be some type of bug with these instances. Need to identify more info associated with these events to see if we can isolate what's happening here

## Check that you can determine language switches from sessions with clicks to new button

In [212]:
#rough query to confirm approach
# will be refine in analysis
query <- 
"
-- sessions where lang button was selected
WITH button AS (
SELECT
    MIN(TO_DATE(dt)) as button_date,
    event.web_session_id as session_id,
    event.skinVersion as skinversion,
    event.isAnon As isAnon,
    event.context as button_type,
    wiki as wiki
FROM event.universallanguageselector
WHERE
    year = 2021
    AND month = 06
    AND Day >= 11
    AND useragent.is_bot = false
    AND event.action = 'compact-language-links-open'
GROUP BY 
    event.web_session_id,
    event.context,
    event.isAnon,
    event.skinversion,
    wiki
),

lang_switches AS (
  SELECT
    TO_DATE(dt) as switch_date,
    event.web_session_id as session_id,
    event.context as switch_context,
    event.timetochangelanguage as switch_time,
    wiki as wiki
FROM event.universallanguageselector
WHERE
    year = 2021
    AND month = 06
    AND Day >= 11
    AND useragent.is_bot = false
    AND event.action = 'language-change'
)

SELECT
    button.button_date,
    button.button_type,
    button.skinversion,
    button.isAnon,
    lang_switches.switch_time,
    lang_switches.switch_date,
    button.session_id,
    button.wiki,
-- sessions with lang switch that occured after button clicks
    IF(lang_switches.session_id IS NOT NULL, 1, 0) AS language_switch,
    lang_switches.switch_context
FROM button
LEFT JOIN lang_switches ON
    button.session_id = lang_switches.session_id
    
"

In [213]:
lang_button_switch_events <-  wmfdata::query_hive(query)

Don't forget to authenticate with Kerberos using kinit



In [214]:
lang_button_switch_bybuttontype <- lang_button_switch_events %>%
    filter(button_type != 'NULL',#filter out non vector events
          language_switch == 1) %>% #only sessions with lang switch
    group_by(button_type, switch_context, language_switch, isanon)  %>%
    summarise(n_sessions = n_distinct(session_id))

lang_button_switch_bybuttontype

`summarise()` regrouping output by 'button_type', 'switch_context', 'language_switch' (override with `.groups` argument)



button_type,switch_context,language_switch,isanon,n_sessions
<chr>,<chr>,<int>,<chr>,<int>
header,interface,1,false,614
other,interface,1,false,2336
other,interface,1,true,77368
other,languages-list,1,false,211
other,languages-list,1,true,5400


Since deployment of the fix on 11 June 2021, were are now recording language switches that occur after the user clicks the new button. These are recorded as context = interface.

There are more language switches that occur using the N more button in sidebar since the new button is currently only deployed to logged-in users that have opt'd into the new vector experience. These numbers appear as expected.


Note: A NULL button_type means the user clicked from a skin type other than vector.



In [215]:
lang_button_switch_byskintype <- lang_button_switch_events %>%
    group_by(button_type, switch_context, language_switch, skinversion)  %>%
    summarise(n_sessions = n_distinct(session_id))

lang_button_switch_byskintype

`summarise()` regrouping output by 'button_type', 'switch_context', 'language_switch' (override with `.groups` argument)



button_type,switch_context,language_switch,skinversion,n_sessions
<chr>,<chr>,<int>,<chr>,<int>
header,interface,1,latest,614
header,NULL,0,latest,220
NULL,interface,1,NULL,3
NULL,NULL,0,NULL,36
other,interface,1,latest,8197
other,interface,1,legacy,71475
other,languages-list,1,latest,5604
other,languages-list,1,legacy,6
other,NULL,0,latest,3076


## Time to Change Language

In [219]:
lang_button_switch_bytime<- lang_button_switch_events  %>%
    filter(language_switch == 1,
           skinversion == "latest") %>%
    group_by(button_type, switch_context, switch_time)  %>%
    summarise(n_sessions = n_distinct(session_id))

lang_button_switch_bytime

`summarise()` regrouping output by 'button_type', 'switch_context' (override with `.groups` argument)



button_type,switch_context,switch_time,n_sessions
<chr>,<chr>,<chr>,<int>
header,interface,10017.0,1
header,interface,10019.0,1
header,interface,10033.900146484375,1
header,interface,100522.89990234375,1
header,interface,10064.39990234375,1
header,interface,10088.89990234375,1
header,interface,10102.900146484375,1
header,interface,101213.60009765625,1
header,interface,10155.900146484375,1


In [ ]:
Confirmed time to change is recorded.

## Check Sidebar Clicks with added skin context fields

In [169]:
query <- 
"
SELECT
    TO_DATE(dt) AS `date`,
    wiki,
    event.web_session_id,
    event.usereditbucket,
    event.skin,
    event.skinVersion,
    event.timetochangelanguage,
    event.isanon,
    event.action,
    event.interfacelanguage,
    event.contentlanguage,
    event.selectedinterfacelanguage,
    Count(*) AS n_events
FROM event.universallanguageselector
WHERE
    year = 2021
    AND month = 06
    AND day >= 11
    AND event.context = 'languages-list'
    AND useragent.is_bot = false
GROUP BY
    TO_DATE(dt),
    wiki,
    event.web_session_id,
    event.usereditbucket,
    event.skin,
    event.skinVersion,
    event.timetochangelanguage,
    event.isanon,
    event.action,
    event.interfacelanguage,
    event.contentlanguage,
    event.selectedinterfacelanguage
"

In [170]:
lang_sidebar_events <-  wmfdata::query_hive(query)

Don't forget to authenticate with Kerberos using kinit



In [171]:
lang_sidebar_events_byskintype<- lang_sidebar_events %>%
    group_by(skin, skinversion, isanon) %>%
    summarize(n_sessions = n_distinct(web_session_id))

lang_sidebar_events_byskintype

`summarise()` regrouping output by 'skin', 'skinversion' (override with `.groups` argument)



skin,skinversion,isanon,n_sessions
<chr>,<chr>,<chr>,<int>
NULL,NULL,false,1
NULL,NULL,true,43
vector,latest,false,2874
vector,latest,true,121084


We are recording clicks to the language list in the sidebar either the latest version of vector or NULL. This is expected as instrumentation for sidebar clicks was limited to language list. Further instrumentation here to clarfiy NULL values would be helpful but it is assumed that any events identifed with a NULL skin type came from non latest vector.

This instrumentation will be good for the AB Test since we will only be looking users on the latest vector skin; however, further clarification of these other skin types will be useful in the future in case we want to know percent of users clicking on these links from legacy vector vs other skin types.

### By Non-Test Wiki

In [172]:
lang_sidebar_events_byskintype_nontestwiki <- lang_sidebar_events %>%
    filter(!wiki %in% c('frwiktionary', 'hewiki', 'ptwikiversity', 'frwiki', 
    'euwiki', 'fawiki', 'ptwiki', 'kowiki', 'trwiki', 'srwiki', 'bnwiki', 'dewikivoyage', 'vecwiki' )) %>%
    group_by(skin, skinversion, isanon) %>%
    summarize(n_sessions = n_distinct(web_session_id))

lang_sidebar_events_byskintype_nontestwiki

`summarise()` regrouping output by 'skin', 'skinversion' (override with `.groups` argument)



skin,skinversion,isanon,n_sessions
<chr>,<chr>,<chr>,<int>
vector,latest,false,5
vector,latest,true,4


In [ ]:
## By Test Wiki

In [173]:
lang_sidebar_events_byskintype_testwiki <- lang_sidebar_events %>%
    filter(wiki %in% c('frwiktionary', 'hewiki', 'ptwikiversity', 'frwiki', 
    'euwiki', 'fawiki', 'ptwiki', 'kowiki', 'trwiki', 'srwiki', 'bnwiki', 'dewikivoyage', 'vecwiki' )) %>%
    group_by(skin, skinversion, isanon) %>%
    summarize(n_sessions = n_distinct(web_session_id))

lang_sidebar_events_byskintype_testwiki

`summarise()` regrouping output by 'skin', 'skinversion' (override with `.groups` argument)



skin,skinversion,isanon,n_sessions
<chr>,<chr>,<chr>,<int>
NULL,NULL,false,1
NULL,NULL,true,43
vector,latest,false,2869
vector,latest,true,121080


Since the AB test has not started, we are not logging a lot of clicks by logged in or logged out users to the lang links on the sidebar on non-test wikis (the new search button was deployed as to all logged-in users on latest vector). 

Note: There are some sessions by both logged-in and logged-out users on the latest vector recorded as having click a lang list link. Not sure where these are coming from but it's such a small percentage it should not impact the AB test.

The majority of clicks occur on test wikis by logged-out as expected. 

Once the AB test runs, any clicks to the lang links in the sidebar should only be recorded by users in the control group. 

### By Action

In [174]:
lang_sidebar_events_byaction<- lang_sidebar_events %>%
    group_by(action) %>%
    summarize(n_sessions = n_distinct(web_session_id))

lang_sidebar_events_byaction

`summarise()` ungrouping output (override with `.groups` argument)



action,n_sessions
<chr>,<int>
language-change,123924


Only associated with `event.action = 'language-change'` as expected.

## Check any follow-up actions

In [175]:
#rough query to confirm approach
# will be refine in analysis
query <- 
"
-- sessions where lang button was selected
WITH button AS (
SELECT
    MIN(TO_DATE(dt)) as button_date,
    event.web_session_id as session_id,
    event.skinVersion as skinversion,
    event.context as button_type,
    wiki as wiki
FROM event.universallanguageselector
WHERE
    year = 2021
    AND month = 06
    AND Day >= 11
    AND useragent.is_bot = false
    AND event.action = 'compact-language-links-open'
GROUP BY 
    event.web_session_id,
    event.context,
    event.skinversion,
    wiki
),

follow_actions AS (
  SELECT
    TO_DATE(dt) as action_date,
    event.action as action_type,
    event.web_session_id as session_id,
    event.context as action_context,
    wiki as wiki
FROM event.universallanguageselector
WHERE
    year = 2021
    AND month = 06
    AND Day >= 11
    AND useragent.is_bot = false
    AND event.action != 'compact-language-links-open'
)

SELECT
    button.button_date,
    button.button_type,
    button.skinversion,
    follow_actions.action_date,
    follow_actions.action_type,
    button.session_id,
    button.wiki,
-- sessions with lang switch that occured after button clicks
    IF(follow_actions.session_id IS NOT NULL, 1, 0) AS follow_action,
    follow_actions.action_context
FROM button
LEFT JOIN follow_actions ON
    button.session_id = follow_actions.session_id
    
"

In [176]:
more_lang_actions <-  wmfdata::query_hive(query)

Don't forget to authenticate with Kerberos using kinit



### By Action Type

In [177]:
# find what types of actions are recorded with a new button clicks
new_button_actions <- more_lang_actions %>%
    filter(button_type == 'header') %>%
    group_by(action_type) %>%
    summarise (n_sessions = n_distinct(session_id))

new_button_actions

`summarise()` ungrouping output (override with `.groups` argument)



action_type,n_sessions
<chr>,<int>
ime-change,3
language-change,219
more-languages-access,1
no-search-results,8
NULL,79
webfonts-enable,1


So far, we have recorded 'ime-change' (user changed the input method), 'no-search-results' (User searched for a language with no results) and 'webfonts-enable' (webfonts-enable: User enabled the webfonts functionality via ULS settings) actions in sessions where clicks to the new button were logged. 

With the fix, we are now also recording language-change events.

### By Action Type and Button Type

In [180]:
# find what types of actions are recorded with all button clicks
all_button_actions <- more_lang_actions %>%
    group_by(action_type, button_type) %>%
    summarise (n_sessions = n_distinct(session_id)) %>%
    arrange(button_type)

all_button_actions

`summarise()` regrouping output by 'action_type' (override with `.groups` argument)



action_type,button_type,n_sessions
<chr>,<chr>,<int>
ime-change,header,3
language-change,header,219
more-languages-access,header,1
no-search-results,header,8
NULL,header,79
webfonts-enable,header,1
language-change,NULL,2
no-search-results,NULL,2
NULL,NULL,13


Some of the actions not recorded for the new clicks button make sense. For example, we shouldn't be recording settings-open events in sessions with clicks to the new language button but I would anticipate there being clicks to switch languages from the button.